# 3. The Raster API: titiler-pgstac

titiler-pgstac is the crown jewel of an eoAPI deployment. You can use titiler-pgstac to serve infinitely customizable visualizations of the data in your catalog with just a few lines of code.

### Dynamic Tiling 101
XYZ map tiles are the backbone of web mapping technology. Traditionally, the tile images are generated from the raw data and stored in cloud storage so they can be easily fetched by a map client in a web browser (or desktop GIS application). These tiles are called "static" tiles because they are generated once and cannot be adjusted unless you regenerate the tiles from the raw source.

The static nature of pre-generated tiles is a problem for dynamic datasets like satellite imagery collections where there may be new data streaming in at all times. It is not efficient to be continually rebuilding static tiles for an entire collection when the data are constantly changing!

**Dynamic tiling** provides a solution to this problem. Instead of pre-rendering all of the tile images for a dynamic dataset, dynamic tiling renders the images on-the-fly as they are requested by client applications! Dynamically rendered tile images will take longer to return to the client but storing the raw data in cloud-optimized data formats will minimize processing latency and you gain the ability to request any visualization configuration (colormap, min/max, band math, expressions, etc).

[titiler](https://developmentseed.org/titiler/) is the state-of-the-art in dynamic tiling. It can take an XYZ tile request, use rasterio/GDAL to read the portion of a data required to render the tile image, then return it to the requester as a .png (or whatever format is requested).

[titiler-pgstac](https://stac-utils.github.io/titiler-pgstac/latest/) extends the core titiler functionality by providing a query interface for a `pgstac` database, allowing users to request visualizations of entire collections or mosaics of items that are in their STAC.

To illustrate the power of titiler-pgstac, consider the following examples:
1. You can create an interactive map with the most recent Sentinel-2 images from your STAC collection in just a few lines of code
2. You can create an interacive map with the most recent **cloud-free** images from any period of time in the archive with 10 lines of code
3. You can view natural color and false color composites of the imagery with one small change

You can do all of this with titiler-pgstac with a single copy of the raw data in cloud storage!

In [1]:
from IPython.display import IFrame

titiler_pgstac_endpoint = "https://helfmwseh8.execute-api.us-west-2.amazonaws.com"

IFrame(
    f"{titiler_pgstac_endpoint}/api.html",
    800,
    600,
)

## 3.1 /collections/{collection_id}
Start out with the most basic titiler-pgstac request: RGB visualization of the most recent images in your Sentinel-2 STAC collection.

To get a visualization of the red/green/blue band combination from your collection you need to tell titiler-pgstac which assets you want using the `assets` query parameter (multiple times in this case).

In [2]:
import json
from urllib.parse import urlencode

import httpx

collection_id = "hrodmn-sentinel-2-c1-l2a"

params = (
    ("assets", "red"),
    ("assets", "green"),
    ("assets", "blue"),
    ("color_formula", "Gamma RGB 3.0 Saturation 1.2 Sigmoidal RGB 15 0.35"),
)

The standard format for distributing information for XYZ tile layers is the `tilejson`. You can request a `tilejson` document from titiler-pgstac and the response will include everything you would need to add the layer to a map.

In [3]:
tilejson_request = httpx.get(
    f"{titiler_pgstac_endpoint}/collections/{collection_id}/WebMercatorQuad/tilejson.json?{urlencode(params, doseq=True)}",
)

print(json.dumps(tilejson_request.json(), indent=2))

{
  "tilejson": "2.2.0",
  "name": "Mosaic for 'hrodmn-sentinel-2-c1-l2a' Collection",
  "version": "1.0.0",
  "scheme": "xyz",
  "tiles": [
    "https://helfmwseh8.execute-api.us-west-2.amazonaws.com/collections/hrodmn-sentinel-2-c1-l2a/tiles/WebMercatorQuad/{z}/{x}/{y}?assets=red&assets=green&assets=blue&color_formula=Gamma+RGB+3.0+Saturation+1.2+Sigmoidal+RGB+15+0.35"
  ],
  "minzoom": 0,
  "maxzoom": 24,
  "bounds": [
    -79.0,
    36.9,
    -75.0,
    40.9
  ],
  "center": [
    -77.0,
    38.9,
    0
  ]
}


The `tiles` key in the tilejson response contains a list of XYZ tile URLs that can be added to most mapping/GIS applications:
  - QGIS
  - ArcGIS
  - Leaflet
  - Mapbox

For now you can take a shortcut to view the map directly in this notebook using the `/map` endpoint which will generate an HTML file with a Leaflet map that can be rendered directly in the notebook.

In [4]:
IFrame(
    f"{titiler_pgstac_endpoint}/collections/{collection_id}/WebMercatorQuad/map?{urlencode(params, doseq=True)}",
    800,
    600,
)

### How does it work?

- titiler-pgstac is running as a Lambda (serverless) function in AWS that started up when you made the request for the `/map` endpoint.
- The `/map` endpoint returns an HTML file that is pre-populated with some map code that includes the layer that you specified with the request parameters
- As you browse the map, the map is sending XYZ tile requests to titiler-pgstac function in AWS
- Each request contains the information titiler-pgstac needs to search for items in the pgstac database and how to construct an image from the items' assets
  - `collection_id`: by specifying the collection ID in the request path you are instructing titiler-pgstac to search for items from a specific STAC collection. Unless otherwise specified, pgstac will retrieve the STAC items in descending order by datetime and it will stop returning results when a tile's geometry is completely covered.
  - `assets`: titiler-pgstac constructs images from STAC assets in the items that match the search
  - by setting `assets=red&assets=blue&assets=green` we are requesting a 3-band rendering from the red, green, and blue assets in each STAC item
  - `color_formula`: since the raw data could have any scale, you may need to apply a color forumula or simpler `rescale` to ensure the output images are visually useful

## 3.2 /searches

Sometimes you will want to render visualizations from a more complex STAC query. titiler-pgstac and pgstac make it possible to pre-register a search query to make it easy to request tiles from a STAC search based on the filter parameters.

The view in the example above may have some clouds visible so you can construct a query that will filter down the STAC item results to items that have low cloud cover.

To do this you can first send a POST request to the `/searches/register` endpoint with your STAC query parameters like you would send to a STAC API when searching for items directly.
Once the search is registered, you can make XYZ tile requests like you did for the default `/collections` view.

In [5]:
# get the bounding box from your STAC collection record to constrain the map view
stac_api_endpoint = "https://pj44p72a3g.execute-api.us-west-2.amazonaws.com"

collection_info = httpx.get(f"{stac_api_endpoint}/collections/{collection_id}").json()

bbox = collection_info["extent"]["spatial"]["bbox"][0]

register_search_request = httpx.post(
    f"{titiler_pgstac_endpoint}/searches/register",
    json={
        "collections": [collection_id],
        "bbox": bbox,
        # filter down to items from March 2025
        "datetime": "2025-03-01T00:00:00Z/2025-04-01T00:00:00Z",
        # with less than 10% cloud cover
        "filter": {
            "op": "lt",
            "args": [
                {"property": "eo:cloud_cover"},
                10,
            ],
        },
    },
    timeout=None,
)

search_response = register_search_request.json()
print(json.dumps(search_response, indent=2))

{
  "id": "96b854a9c49919608e68e0f4462eadf3",
  "links": [
    {
      "href": "https://helfmwseh8.execute-api.us-west-2.amazonaws.com/searches/96b854a9c49919608e68e0f4462eadf3/info",
      "rel": "metadata",
      "title": "Mosaic metadata"
    },
    {
      "href": "https://helfmwseh8.execute-api.us-west-2.amazonaws.com/searches/96b854a9c49919608e68e0f4462eadf3/{tileMatrixSetId}/tilejson.json",
      "rel": "tilejson",
      "templated": true,
      "title": "Link for TileJSON (Template URL)"
    },
    {
      "href": "https://helfmwseh8.execute-api.us-west-2.amazonaws.com/searches/96b854a9c49919608e68e0f4462eadf3/{tileMatrixSetId}/map",
      "rel": "map",
      "templated": true,
      "title": "Link for Map viewer (Template URL)"
    },
    {
      "href": "https://helfmwseh8.execute-api.us-west-2.amazonaws.com/searches/96b854a9c49919608e68e0f4462eadf3/{tileMatrixSetId}/WMTSCapabilities.xml",
      "rel": "wmts",
      "templated": true,
      "title": "Link for WMTS (Template U

The response comes back with an `id` which uniquely identifies this search and a handful of useful links associated with our newly registered search.

Now you can browse the results of this search with the `/map` endpoint like you did earlier.

In [6]:
search_id = search_response["id"]

params = (
    ("assets", "red"),
    ("assets", "green"),
    ("assets", "blue"),
    ("color_formula", "Gamma RGB 3.0 Saturation 1.2 Sigmoidal RGB 15 0.35"),
)

IFrame(
    f"{titiler_pgstac_endpoint}/searches/{search_id}/WebMercatorQuad/map?{urlencode(params, doseq=True)}",
    900,
    600,
)

Et voila! Now you have a view of the most recent images from your STAC collection that had less than 10% cloud cover. This works because we have so much useful information packed into the STAC items - thank goodness for STAC metadata!

What other kinds of filters would be useful to apply?

## 3.3 band math expressions

Perhaps you want to view NDVI or some other band math expression. You can set `asset_as_band=True` then write a mathematical expression that uses the asset keys as variables which titiler-pgstac will evaluate for each tile.

In the example below you can request tiles for a view of NDVI and specify the colormap from a [list](https://cogeotiff.github.io/rio-tiler/colormap/#intervals-colormaps) of possible values. The results will be calculated on-the-fly by titiler-pgstac for each XYZ tile request, and returned to your map client in this notebook.

In [7]:
params = (
    ("asset_as_band", "True"),
    ("expression", "(nir - red) / (nir + red)"),
    ("colormap_name", "viridis"),
    ("rescale", "-0.5,1"),
)

IFrame(
    f"{titiler_pgstac_endpoint}/collections/{collection_id}/WebMercatorQuad/map?{urlencode(params, doseq=True)}",
    800,
    600,
)

## 3.4 Customization

eoAPI ships with the default titiler-pgstac application out-of-the-box, but there are several configuration options that you can set using environment variables in the API runtime environment. If there are features or other endpoints (like MosaicJSON capability) that you want to add, you will need to define a custom runtime that extends the default application.

See [maap-eoapi](https://github.com/MAAP-Project/maap-eoapi/tree/main/cdk/runtimes/eoapi/raster/eoapi/raster) for an example of a custom runtime that extends the default settings.

### 3.4.1 /external

Sometimes you may want to render visualizations of data that are **not** cataloged in your STAC. The `/external` family of endpoints can be enabled by setting the environment variable `TITILER_PGSTAC_API_ENABLE_EXTERNAL_DATASET_ENDPOINTS=True`. This will enable you to create tile endpoints for any file available over HTTP.